# Scraping investor's portfolio data  from Dataroma

In this post we will provide the code to get the portfolio data from dataroma.com.

In [1]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup

In [2]:
my_url = "https://www.dataroma.com/m/home.php"

In [3]:
my_url = urllib.request.urlopen(my_url)

In [4]:
soup = BeautifulSoup(my_url.read(), 'html.parser')

In [5]:
links = []
data = []

for t in soup.find_all('li'):
    for a in t.find_all('a'):
        links.append(a['href'])
        
for t in soup.find_all('li'):
    for a in t.find_all('a'):
        data.append(a.text)

In [6]:
links = ['https://dataroma.com' + l for l in links]

raw_df = pd.DataFrame({'name':data,
              'links':links})

In [7]:
raw_df = raw_df.iloc[8:,:]

In [8]:
raw_df.head()

,name,links
8,David Rolfe - Wedgewood Partners Updated 14 Fe...,https://dataroma.com/m/holdings.php?m=WP
9,Nelson Peltz - Trian Fund Management Updated 1...,https://dataroma.com/m/holdings.php?m=TF
10,Jeffrey Ubben - ValueAct Capital Updated 14 F...,https://dataroma.com/m/holdings.php?m=VA
11,Eddie Lampert - RBS Partners Updated 14 Feb 2020,https://dataroma.com/m/holdings.php?m=EL
12,Bill & Melinda Gates Foundation Trust Updated ...,https://dataroma.com/m/holdings.php?m=GFT


In [9]:
raw_df[['investor_name', 'update_date']] = raw_df['name'].str.split('Updated', expand = True)

In [10]:
raw_df[['investor_name', 'fund_name', '_']] = raw_df['investor_name'].str.split('-', expand = True)

In [11]:
raw_df = raw_df[['investor_name', 'fund_name', 'links']]

In [12]:
raw_df.head()

,investor_name,fund_name,links
8,David Rolfe,Wedgewood Partners,https://dataroma.com/m/holdings.php?m=WP
9,Nelson Peltz,Trian Fund Management,https://dataroma.com/m/holdings.php?m=TF
10,Jeffrey Ubben,ValueAct Capital,https://dataroma.com/m/holdings.php?m=VA
11,Eddie Lampert,RBS Partners,https://dataroma.com/m/holdings.php?m=EL
12,Bill & Melinda Gates Foundation Trust,None,https://dataroma.com/m/holdings.php?m=GFT


Now we get the data for individual investor.

In [13]:
raw_df[raw_df['investor_name'].str.contains('Warren')]['links'].values.tolist()

['https://dataroma.com/m/holdings.php?m=BRK']

In [14]:
raw_df['investor_name'].values

array(['David Rolfe ', 'Nelson Peltz ', 'Jeffrey Ubben ',
       'Eddie Lampert ', 'Bill & Melinda Gates Foundation Trust ',
       'Stephen Mandel ', 'David Abrams ', 'David Einhorn ',
       'Bruce Berkowitz ', 'Bill Ackman ', 'David Tepper ', 'Carl Icahn ',
       'Warren Buffett ', 'Alex Roepers ', 'Chuck Akre ', 'Allan Mecham ',
       'Michael Price ', 'Bill Miller ', 'Pat Dorsey ', 'Leon Cooperman ',
       'Lee Ainslie ', 'Howard Marks ', 'Prem Watsa ', 'Glenn Greenberg ',
       'Chris Hohn ', 'Terry Smith ', 'Daniel Loeb ', 'Seth Klarman ',
       'Christopher Bloomstran ', 'Third Avenue Management ',
       'Mohnish Pabrai ', 'Jefferies ', 'Robert Olstein ',
       'Thomas Russo ', 'Thomas Gayner ', 'William Von Mueffling ',
       'Kahn Brothers Advisors ', 'Wallace Weitz ', 'Tweedy Browne Co. ',
       'Guy Spier ', 'Sam Peters ', 'Dodge & Cox ', 'Richard Pzena ',
       'Mairs & Power ', 'Steven Romick ', 'FPA ', 'Mark Hillman ',
       'John Rogers ', 'Charles Bobrinskoy

These are all the invetor names available.

Lets try Bill Gates.

In [15]:
raw_df[raw_df['investor_name'].str.contains('Bill')]

,investor_name,fund_name,links
12,Bill & Melinda Gates Foundation Trust,None,https://dataroma.com/m/holdings.php?m=GFT
17,Bill Ackman,Pershing Square Capital Management,https://dataroma.com/m/holdings.php?m=psc
25,Bill Miller,Miller Value Partners,https://dataroma.com/m/holdings.php?m=LMM
64,Bill Nasgovitz,Heartland Select Value,https://dataroma.com/m/holdings.php?m=HRSVX
67,Bill Nygren,Oakmark Select,https://dataroma.com/m/holdings.php?m=oaklx


That returned more than one value. Lets Try Bill & Melinda

In [16]:
raw_df[raw_df['investor_name'].str.contains('Bill & Melinda')]

,investor_name,fund_name,links
12,Bill & Melinda Gates Foundation Trust,None,https://dataroma.com/m/holdings.php?m=GFT


In [17]:
port_data = [urllib.request.urlopen(a) for a in raw_df[raw_df['investor_name'].str.contains('Bill & Melinda')]['links'].values.tolist()]

In [18]:
for i in port_data:
    soup = BeautifulSoup(i.read(),'html.parser')

In [19]:
tbody = soup.find('tbody')

In [20]:
data = []

for d in tbody:
    data.append(d.text.split('\n'))

In [21]:
df = pd.DataFrame(data)

In [22]:
df.head(10)

,0,1,2,3,4,5,6,7,8
0,,≡,BRK.B - Berkshire Hathaway CL B,52.94,"49,912,525",Reduce 9.11%,$226.50,"$11,305,187,000",
1,,≡,WM - Waste Management Inc.,9.94,"18,633,672",,$113.96,"$2,123,493,000",
2,,≡,CAT - Caterpillar Inc.,7.79,"11,260,857",,$147.68,"$1,663,003,000",
3,,≡,CNI - Canadian Natl Railway Co.,7.25,"17,126,874",,$90.45,"$1,549,126,000",
4,,≡,WMT - Wal-Mart Stores,6.46,"11,603,000",,$118.84,"$1,378,901,000",
5,,≡,ECL - Ecolab Inc.,3.95,"4,366,426",,$192.99,"$842,677,000",
6,,≡,CCI - Crown Castle International Corp.,3.55,"5,332,900",,$142.15,"$758,072,000",
7,,≡,UPS - United Parcel Service,2.48,"4,525,329",,$117.06,"$529,735,000",
8,,≡,FDX - FedEx Corp.,2.14,"3,024,999",,$151.21,"$457,410,000",
9,,≡,KOF - Coca-Cola FEMSA S A CV,1.76,"6,214,719",,$60.62,"$376,736,000",


In [23]:
df = df.drop(columns=[0,1, 8])

In [24]:
df.columns = ['Stock', 'portfolio_percent', 'shares', 'recent', 'reported_price', 'value']

In [25]:
df[['symbol', 'company']] = df['Stock'].str.split(r' - ',expand = True)

In [26]:
df = df[['symbol', 'company', 'portfolio_percent', 'shares', 'recent', 'reported_price', 'value']]

In [27]:
df

,symbol,company,portfolio_percent,shares,recent,reported_price,value
0,BRK.B,Berkshire Hathaway CL B,52.94,"49,912,525",Reduce 9.11%,$226.50,"$11,305,187,000"
1,WM,Waste Management Inc.,9.94,"18,633,672",,$113.96,"$2,123,493,000"
2,CAT,Caterpillar Inc.,7.79,"11,260,857",,$147.68,"$1,663,003,000"
3,CNI,Canadian Natl Railway Co.,7.25,"17,126,874",,$90.45,"$1,549,126,000"
4,WMT,Wal-Mart Stores,6.46,"11,603,000",,$118.84,"$1,378,901,000"
5,ECL,Ecolab Inc.,3.95,"4,366,426",,$192.99,"$842,677,000"
6,CCI,Crown Castle International Corp.,3.55,"5,332,900",,$142.15,"$758,072,000"
7,UPS,United Parcel Service,2.48,"4,525,329",,$117.06,"$529,735,000"
8,FDX,FedEx Corp.,2.14,"3,024,999",,$151.21,"$457,410,000"
9,KOF,Coca-Cola FEMSA S A CV,1.76,"6,214,719",,$60.62,"$376,736,000"


---
#### Create a function

In [28]:
def get_investor_portfolio(name):
    
    n = name.title()
    
    port_data = [urllib.request.urlopen(a) for a in raw_df[raw_df['investor_name'].str.contains(n)]['links'].values.tolist()]
    
    for i in port_data:
        soup = BeautifulSoup(i.read(),'html.parser')
    
    tbody = soup.find('tbody')
    
    data = []

    for d in tbody:
        data.append(d.text.split('\n'))
        
    df = pd.DataFrame(data)
    
    df = df.drop(columns=[0,1, 8])
    df.columns = ['Stock', 'portfolio_percent', 'shares', 'recent', 'reported_price', 'value']
    
    df[['symbol', 'company']] = df['Stock'].str.split(r' - ',expand = True)
    
    df = df[['symbol', 'company', 'portfolio_percent', 'shares', 'recent', 'reported_price', 'value']]
    
    return df

Lets try our function.

First lets get the Portfolio for Bill Ackman

In [29]:
get_investor_portfolio('Ackman')

,symbol,company,portfolio_percent,shares,recent,reported_price,value
0,CMG,Chipotle Mexican Grill Inc.,22.03,"1,724,310",,$837.11,"$1,443,437,000"
1,HLT,Hilton Worldwide Holdings,17.87,"10,556,805",,$110.91,"$1,170,855,000"
2,LOW,Lowe's Cos.,15.74,"8,613,212",,$119.76,"$1,031,518,000"
3,QSR,Restaurant Brands International,14.68,"15,084,304",,$63.77,"$961,926,000"
4,BRK.B,Berkshire Hathaway CL B,13.88,"4,015,594",,$226.50,"$909,532,000"
5,SBUX,Starbucks Corp.,7.32,"5,457,954",Reduce 41.40%,$87.92,"$479,863,000"
6,HHC,Howard Hughes Corp.,4.25,"2,197,389",Add 83.91%,$126.80,"$278,629,000"
7,A,Agilent Technologies,4.23,"3,248,114",Add 11.39%,$85.31,"$277,097,000"


Now lets try Daniel Loeb

In [30]:
get_investor_portfolio('Loeb')

,symbol,company,portfolio_percent,shares,recent,reported_price,value
0,BAX,Baxter International Inc.,16.83,"17,500,000",Add 3.86%,$83.62,"$1,463,350,000"
1,UTX,United Technologies,8.97,"5,207,000",Reduce 13.22%,$149.76,"$779,800,000"
2,CPB,Campbell Soup,8.19,"14,400,000",Reduce 15.29%,$49.42,"$711,648,000"
3,AGN,Allergan Plc,7.92,"3,600,000",,$191.17,"$688,212,000"
4,DHR,Danaher Corp.,6.55,"3,710,000",,$153.48,"$569,411,000"
5,CRM,Salesforce.com,4.12,"2,200,000",Reduce 12.00%,$162.64,"$357,808,000"
6,FIS,Fidelity National Information Services,4.00,"2,500,000",Add 0.00%,$139.09,"$347,725,000"
7,BURL,Burlington Stores Inc.,3.67,"1,400,000",Reduce 6.67%,$228.03,"$319,242,000"
8,ADBE,Adobe Inc.,3.60,"950,000",Add 15.15%,$329.81,"$313,320,000"
9,IQV,IQVIA Holdings Inc.,3.55,"2,000,000",,$154.51,"$309,020,000"


Finally lets try Warrne Buffett.

In [31]:
get_investor_portfolio('Buffett')

,symbol,company,portfolio_percent,shares,recent,reported_price,value
0,AAPL,Apple Inc.,29.74,"245,155,566",Reduce 1.48%,$293.65,"$71,989,933,000"
1,BAC,Bank of America Corp.,13.46,"925,008,600",Reduce 0.24%,$35.22,"$32,578,803,000"
2,KO,Coca Cola Co.,9.15,"400,000,000",,$55.35,"$22,140,000,000"
3,AXP,American Express,7.80,"151,610,700",,$124.49,"$18,874,016,000"
4,WFC,Wells Fargo,7.18,"323,212,918",Reduce 14.58%,$53.80,"$17,388,855,000"
5,KHC,Kraft Heinz Co.,4.32,"325,634,818",,$32.13,"$10,462,647,000"
6,JPM,JPMorgan Chase & Co.,3.43,"59,514,932",,$139.40,"$8,296,382,000"
7,USB,U.S. Bancorp,3.24,"132,459,618",,$59.29,"$7,853,532,000"
8,MCO,Moody's Corp.,2.42,"24,669,778",,$237.41,"$5,856,852,000"
9,DAL,Delta Air Lines Inc.,1.71,"70,910,456",,$58.48,"$4,146,844,000"
